# Supervised Learning- Regression Models

## An Introduction to Algorithmic Trading

---

## Overview

Welcome to this Excercise. We are now going to use our new skills to build our first unsupervised learning models. In particular, we are going to use multiple methods to create a number unsupervisoed learning models such kmeans for clustering and PCA to reduce the dimensionality of or data.

---

## Introduction

We are going to start with a slightly modified version of our previously used datasets. As always we start with a universe of stock prices for the the S&P 500.

---

## Key Features

1) First use the yfinance library to download both data sets. You will be using the constituents of the S&P 500. 

2) We will then start to compute returns.



In [1]:
# We can decide whther we want to download the data or use the saved csv version of it
DOWNLOAD_DATA_FROM_API = False 
MIN_REQUIRED_NUM_OBS_PER_TICKER=100

In [2]:
# Import Libraries
import yfinance as yf
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
OMP_NUM_THREADS=2
import matplotlib.pyplot as plt

In [3]:

if DOWNLOAD_DATA_FROM_API == True:
    # Get the list of S&P 500 constituents
    sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()
    
    # Filter out Class B shares that have a '.B' in the ticker name
    sp500_tickers = [ticker for ticker in sp500_tickers if '.B' not in ticker]
    
    # Define the start and end dates for historical data
    start_date = '2000-01-01'
    end_date   = '2024-05-01'
    
    # Download historical prices for the list of ticker sp500_tickers
    historical_prices = yf.download(sp500_tickers, start=start_date, end=end_date)

    # Remove the MultiIndex and keep only the second level
    historical_prices.columns = historical_prices.columns.droplevel(0)
    
    # Filter and keep only columns where the first level of the MultiIndex is 'Adj Close'
    historical_prices  = historical_prices.loc[:, historical_prices.columns.get_level_values(0) == 'Adj Close']

    # Count non-missing values for each ticker
    ticker_counts = historical_prices.count()

    # Filter out tickers with fewer than n=MIN_REQUIRED_NUM_OBS_PER_TICKER=100 non-missing values
    valid_tickers = ticker_counts[ticker_counts >= MIN_REQUIRED_NUM_OBS_PER_TICKER].index
    
    # Filter the DataFrame based on valid tickers
    historical_prices = historical_prices[valid_tickers]
    

else:
    # Read the previously download data
    historical_prices = pd.read_csv('historical_prices.csv', index_col='Date', parse_dates=True)
    historical_prices.columns.name = 'Ticker'

In [4]:
# Count non-missing values for each ticker
ticker_counts = historical_prices.count()

# Filter out tickers with fewer than n=MIN_REQUIRED_NUM_OBS_PER_TICKER=100 non-missing values
valid_tickers = ticker_counts[ticker_counts >= MIN_REQUIRED_NUM_OBS_PER_TICKER].index

# Filter the DataFrame based on valid tickers
historical_prices = historical_prices[valid_tickers]
    

In [5]:
# Print the first 5 rows
historical_prices.head()


Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,43.613007,NaN,0.844981,NaN,NaN,8.992848,1.277778,NaN,16.274675,28.438286,...,NaN,11.505342,NaN,6.977997,18.328693,NaN,4.680301,NaN,25.027779,NaN
2000-01-04,40.281452,NaN,0.773741,NaN,NaN,8.735912,1.270833,NaN,14.909400,26.999619,...,NaN,11.073115,NaN,7.138673,17.977634,NaN,4.586222,NaN,24.666668,NaN
2000-01-05,37.782795,NaN,0.785063,NaN,NaN,8.719849,1.388889,NaN,15.204174,27.393782,...,NaN,11.659699,NaN,7.414120,18.957697,NaN,4.609740,NaN,25.138889,NaN
2000-01-06,36.344185,NaN,0.717125,NaN,NaN,9.024967,1.375000,NaN,15.328290,26.644875,...,NaN,12.205125,NaN,7.345260,19.937763,NaN,4.570544,NaN,23.777779,NaN
2000-01-07,39.372852,NaN,0.751094,NaN,NaN,9.121321,1.451389,NaN,16.072987,27.393782,...,NaN,11.803776,NaN,7.345260,19.879248,NaN,4.468626,NaN,23.513889,NaN


In [8]:
def computingReturns(close_prices,list_of_momentums): 
    forecast=1        
    f_returns = close_prices.pct_change(forecast)            
    f_returns = f_returns.shift(-forecast)
    f_returns = pd.DataFrame(f_returns.unstack())
    name = "F_"+str(forecast)+"_d_returns"
    f_returns.rename(columns={0: name}, inplace = True)
    f_returns.reset_index(inplace = True)
    f_returns.rename(columns={'level_0':'Ticker'}, inplace=True)
    # We add the forward returns to total_returns
    total_returns = f_returns
    
    for i in list_of_momentums:   
        feature = close_prices.pct_change(i)
        feature = pd.DataFrame(feature.unstack())
        name = str(i)+"_d_returns"        
        feature.reset_index(inplace = True)
        feature.rename(columns={0: name, 'level_0':'Ticker'}, inplace = True)
        # We add each 
        total_returns = pd.merge(total_returns,feature,left_on=['Ticker', 'Date'],right_on=['Ticker', 'Date'], how='left', suffixes=('_original', 'right'))
      
    total_returns.dropna(axis=0, how='any', inplace=True) 
    total_returns.set_index(['Date', 'Ticker'], inplace=True)

    return total_returns

In [9]:
#We can choose how many momentums and which ones we want to create
list_of_momentums = [1,2,3,4,5,10,15,20] # [1,2,3,4,5,10].
#list_of_momentums = []
total_data = computingReturns(historical_prices, list_of_momentums)
total_data.head()

,,F_1_d_returns,1_d_returns,2_d_returns,3_d_returns,4_d_returns,5_d_returns,10_d_returns,15_d_returns,20_d_returns
Date,Ticker,,,,,,,,,
2000-02-01,A,0.077465,0.072710,0.043159,0.037443,0.039341,0.048938,-0.006993,0.029918,-0.013889
2000-02-02,A,0.016340,0.077465,0.155807,0.123967,0.117808,0.119854,0.092857,0.125000,0.150376
2000-02-03,A,-0.019293,0.016340,0.095070,0.174693,0.142332,0.136073,0.141284,0.166979,0.246493
2000-02-04,A,0.042623,-0.019293,-0.003268,0.073944,0.152030,0.120294,0.109091,0.127541,0.270833
2000-02-07,A,-0.006289,0.042623,0.022508,0.039215,0.119718,0.201133,0.160584,0.162706,0.223077


Let's verify the correctness of returns for ticker A:

To get the first row of returns:
                    F_1_d_returns  1_d_returns
2000-01-04 	MMM 	0.028966 	   -0.039735


2000-01-03 00:00:00 	17.557297
2000-01-04 00:00:00 	16.859657
2000-01-05 00:00:00 	17.348011 	

To achieve F_1_d_returns we need to:

buy at : 2000-01-04 00:00:00 	16.859657
sell at: 2000-01-05 00:00:00 	17.348011 	

(17.348011 -16.859657)/16.859657 = 0.028965832460292704

To achieve 1_d_returns we need to:

buy at : 2000-01-03 00:00:00 	17.557297
sell at: 2000-01-04 00:00:00 	16.859657

(16.859657 -17.557297)/17.557297 = -0.03973504577612373